In [1]:
from eda_import import *

In [2]:
import pysam

In [3]:
print(datetime.datetime.today().date())

2018-05-08


In [4]:
TOTAL_NUM_CALLS = 67544140

In [5]:
%time adf = pd.read_pickle('./results/all_cba.KLEAT.on-target-cleaned.pkl')

CPU times: user 1.19 s, sys: 896 ms, total: 2.09 s
Wall time: 2.81 s


In [6]:
ON_TARGET_NUM_CALLS = 11489199
assert adf.shape[0] == ON_TARGET_NUM_CALLS

# Redo hexamer search

In [7]:
adf2 = adf.copy()

In [8]:
# convert 1-based to 0-based
adf2['clv0'] = adf2['clv'] - 1

In [9]:
REF_FA = './reference_data/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly.fa'
refseq = pysam.FastaFile(REF_FA)

In [10]:
clv0s_df = adf2[['seqname', 'gene_name', 'clv0', 'strand']].drop_duplicates()

In [11]:
assert clv0s_df.shape[0] == 459113

In [12]:
# hexamer_search code is part of tasrkleat, https://github.com/bcgsc/tasrkleat/
from hexamer_search import search_hexamer

def search_hexamer_wrapper(refseq, chrm, clv, strand, window=50):
    chrm = chrm.replace('chr', '')
    res = search_hexamer.search(refseq, chrm, clv, strand, window)
    if res is None:
        res = ['NA', -1, -1]
    return pd.Series(res, index=['hexamer', 'hexamer_id', 'hexamer_loc0'])

In [13]:
%%time
hexm_df = clv0s_df.apply(
    lambda row: search_hexamer_wrapper(refseq, row.seqname, row.clv0, row.strand), 
    axis=1)

CPU times: user 3min 8s, sys: 6.72 s, total: 3min 15s
Wall time: 3min 15s


In [14]:
# convert to 1-based
%time hexm_df['hexamer_loc'] = hexm_df.hexamer_loc0.apply(lambda v: v + 1 if v != -1 else v)

CPU times: user 233 ms, sys: 9.09 ms, total: 242 ms
Wall time: 239 ms


In [15]:
hexm_df.head(2)

,hexamer,hexamer_id,hexamer_loc0,hexamer_loc
0,NA,-1,-1,-1
1,TTTAAA,14,100732435,100732436


In [16]:
hexm_df.shape

(459113, 4)

In [17]:
hexm_ndf = pd.concat([clv0s_df, hexm_df], axis=1)

In [18]:
hexm_ndf.shape

(459113, 8)

In [19]:
adf3 = adf2.merge(hexm_ndf, on=['seqname', 'gene_name', 'clv0', 'strand'], how='left')

In [20]:
assert adf3.shape[0] == adf.shape[0]

In [21]:
adf4 = adf3.drop(['clv0', 'hexamer_loc0'], axis=1)

In [22]:
adf4.head(2)

,gene_name,strand,seqname,clv,length_of_tail_in_contig,number_of_bridge_reads,max_bridge_read_tail_length,analysis_id,hexamer,hexamer_id,hexamer_loc
0,RTCA,+,chr1,100732395,0,0,0,1d811d54-caa1-41f3-bfc2-400eebdd97ef,NA,-1,-1
1,RTCA,+,chr1,100732476,0,0,0,1d811d54-caa1-41f3-bfc2-400eebdd97ef,TTTAAA,14,100732436


In [23]:
%time adf3.to_pickle('./results/all_cba.KLEAT.on-target-cleaned-hexamer-researched.pkl')

CPU times: user 0 ns, sys: 28.2 s, total: 28.2 s
Wall time: 37.8 s
